# Predicting heart disease using machine learning

## Problem definition:
> Given clinical parameters about a patient can we predict if that person has heart disease?

## Data:
> Can be found [here](https://www.kaggle.com/ronitf/heart-disease-uci).

> This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them. In particular, the Cleveland database is the only one that has been used by ML researchers to
this date. The "goal" field refers to the presence of heart disease in the patient. It is integer valued from 0 (no presence) to 4.

#### Features:
* age: age
* sex: sex
* cp: chest pain type (4 values)
* trestbps: resting blood pressure
* chol: serum cholestoral in mg/dl
* fbs: fasting blood sugar > 120 mg/dl
* restecg: resting electrocardiographic results (values 0,1,2)
* thalach: maximum heart rate achieved
* exang: exercise induced angina
* oldpeak: oldpeak = ST depression induced by exercise relative to rest
* slope: the slope of the peak exercise ST segment
* ca: number of major vessels (0-3) colored by flourosopy
* thal: thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

## Evaluation:
> If the model can reach 90+% accuracy, then it'll be feasable to integrate in an application.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np